In [0]:
-- STEP 1: Validation
SELECT
  CASE
    WHEN COUNT(*) = 0 THEN 'Employees table is empty'
    ELSE 'Validation passed'
  END as message
FROM employees;

-- STEP 2 + 3: Merge cleaned data
MERGE INTO employees_clean t
USING (
    SELECT
        EmployeeID,
        Name,
        Department,
        Role,Salary,Experience,Location
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY EmployeeID ORDER BY EmployeeID) rn
        FROM employees
    )
    WHERE rn = 1
) s
ON t.EmployeeID = s.EmployeeID

WHEN MATCHED AND (
    t.Name <> s.Name OR
    t.Department <> s.Department OR
    t.Role <> s.Role OR
    t.Salary <> s.Salary OR
    t.Experience <> s.Experience OR
    t.Location <> s.Location)
THEN
  UPDATE SET
    t.Name = s.Name,
    t.Department = s.Department,
    t.Role = s.Role,
    t.Salary = s.Salary,
    t.Experience = s.Experience,
    t.Location = s.Location

WHEN NOT MATCHED THEN
  INSERT (EmployeeID, Name, Department,Role, Salary, Experience,Location)
  VALUES (s.EmployeeID, s.Name, s.Department, s.Role, s.Salary,s.Experience,s.Location);
